# Initial Upload of Data

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from dotenv import load_dotenv
dotenv = load_dotenv()
POSTGRESQL_CONNECTION_STRING = os.getenv("POSTGRESQL_CONNECTION_STRING")

In [3]:
from database import (
    Base,
    session_engine_from_connection_string,
    User,
    Item,
    Bin,
    convert_df_to_lst_of_table_objects
)

## Create DB Tables

In [4]:
# create session and engine
session, engine = session_engine_from_connection_string(POSTGRESQL_CONNECTION_STRING)

conn = engine.connect()

# tables to be created
table_objects = [
    User.__table__,
    Item.__table__,
    Bin.__table__,
]

# Drop All Tables
Base.metadata.drop_all(engine, table_objects)

# Create All Tables
Base.metadata.create_all(engine, table_objects)
# session.close()

## Load Data into Items

In [5]:
data_folder = os.path.join(os.getcwd(), "data")
file_name = "items.csv"
item_file_path = os.path.join(data_folder, file_name)
df_item = pd.read_csv(item_file_path)

In [6]:
df_item.rename(columns={
    "name": "item_name",
    "disposal": "disposal_instruction",
    "imageUrl": "image_url",
    "instruction": "additional_instruction",
    "recyclable": "is_recyclable"
}, inplace=True)

df_item.drop(columns=["similarItem"], inplace=True)
df_item["num_browsed"] = 0

In [7]:
item_obj = convert_df_to_lst_of_table_objects(df_item, Item)
session.add_all(item_obj)
session.commit()